In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'

--2021-05-11 12:39:51--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.128, 108.177.11.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   203MB/s    in 9.5s    

2021-05-11 12:40:00 (201 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [ ]:
!pip install wandb
import wandb
!wandb login 781ea05d3016dc46e417672b13ee6d7047c00562
import random
import numpy as np
from importlib import reload
import RNN
RNN = reload(RNN)
from wandb.keras import WandbCallback

# Path to the data txt file on disk.
train_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"
# open and save the files to lists
with open(train_data, "r", encoding="utf-8") as f:
    train_lines = f.read().split("\n")
with open(val_data, "r", encoding="utf-8") as f:
    val_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
train_lines.pop()
val_lines.pop()
random.shuffle(train_lines)
print(train_lines[0:2])

# embedding pre processing
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# go through the train lines and split them into 3 and save input and target
for line in train_lines[: (len(train_lines) - 1)]:
    # because we want english to devanagiri conversion
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    # append it to the main input texts list
    input_texts.append(input_text)
    # append it to the main target texts list
    target_texts.append(target_text)
    # to find the number of unique characters in both
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
# add the space character to both
input_characters.add(" ")
target_characters.add(" ")
# sort it
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
# find the number
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# find the maximum length of input word and target word
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
# create an index
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
print((input_token_index))
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print((target_token_index))
# create an 0 array for encoder input size of (input_texts,max_seqlen,tokens)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
# create decoder input
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
# create decoder target
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
# for each sample convert it into character encoding i.e. if
# at that position a character is present then encode the index of that character there
# this is done for both encoder and decoder input data for further word embedding
# but target data is one hot encoded.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    # remaining positions set as empty space
    encoder_input_data[i, t + 1:] = input_token_index[" "]
    # similarly do for decoder data
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        # check if t >0 since decoder targer data is ahead
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # append both the remaining positions of both the datas with empty space
    decoder_input_data[i, t + 1:] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# embedding validation
# for validation data, almost same
val_input_texts = []
val_target_texts = []
for line in val_lines[: (len(val_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)
val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])
val_encoder_input_data = np.zeros(
    (len(val_input_texts), val_max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), val_max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), val_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1:] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def sweep():
  run = wandb.init()
  config = wandb.config
  name = ('cell_type=%s' % config.cell_type) + "_" \
          + ('embedding=%d' % config.embedding_size) + "_" \
          + ('n_enc_layers=%d' % config.n_encoder_layers) + "_" \
          + ('n_dec_layers=%d' % config.n_decoder_layers) + "_" \
          + ('latent=%d' % config.latent_dimension) + "_" \
          + ('dropout=%.2f' % config.dropout) + "_" \
          + ('epochs=%d' % config.epochs)

  wandb.run.name = name
  batch_size = 64  # Batch size for training.
  epochs = config.epochs  # Number of epochs to train for.
  latent_dim = config.latent_dimension  # Latent dimensionality of the encoding space. #hidden states hyperparameter
  
  # create RNN model
  model = RNN.RNN(embedding_size=config.embedding_size, n_encoder_tokens=num_encoder_tokens, n_decoder_tokens=num_decoder_tokens,
                  n_encoder_layers=config.n_encoder_layers, n_decoder_layers=config.n_decoder_layers, latent_dimension=latent_dim,
                  cell_type=config.cell_type, target_token_index=target_token_index, max_decoder_seq_length=max_decoder_seq_length,
                  reverse_target_char_index=reverse_target_char_index, dropout=config.dropout)
  model.fit(encoder_input_data, decoder_input_data, decoder_target_data,
            batch_size, epochs, callbacks=[WandbCallback()]
            )  
  val_accuracy = model.accuracy(val_encoder_input_data, val_target_texts)
  wandb.log({'val_accuracy': val_accuracy})
  run.finish()
  
# run sweeps
sweep_config = {
    'method': 'bayes',  # grid, random
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [20,25,30]
        },
        'embedding_size': {
            'values': [16,32,64,128,256]
        },
        'n_encoder_layers': {
            'values': [1,2,3]
        },
        'n_decoder_layers': {
            'values': [1,2,3]
        },
        'latent_dimension': {
            'values': [16, 32, 64, 256, 512]
        },
        'cell_type': {
            'values': ['gru', 'lstm', 'rnn']
        },                             
        'dropout': {
            'values': [0.0,0.2,0.3,0.4,0.5]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="cs6910-assignment3-sweep-tamil")

# %% start wandb sweep
# wandb.agent(sweep_id, sweep)
#wandb.agent('k6ivgfhr', sweep, project="cs6910-assignment2-sweep")
wandb.agent(sweep_id, sweep, project="cs6910-assignment3-sweep-tamil")

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 133kB 34.3MB/s 
     |████████████████████████████████| 163kB 27.4MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0ce03048d113c5696f4bb42294134f051e9db706cac07d1f9552fb97f4f58f89
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=067267acd72001b363b31cfb02daba653e6ce03b943e172426337a8617f1f612
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
['விரும்புவார்கள்\tvirumbhuvaargal\t1', 'இடம்பெற்றுள்ள\tidampetrtrulla\t1']
N

wandb: Agent Starting Run: s17trzjo with config:
wandb: 	cell_type: gru
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	epochs: 30
wandb: 	latent_dimension: 32
wandb: 	n_decoder_layers: 3
wandb: 	n_encoder_layers: 3
wandb: Currently logged in as: akceg (use `wandb login --relogin` to force relogin)


Epoch 1/30
1066/1066 [==============================] - 38s 15ms/step - loss: 1.2213 - accuracy: 0.7038
Epoch 2/30
1066/1066 [==============================] - 15s 14ms/step - loss: 0.7848 - accuracy: 0.7688
Epoch 3/30
1066/1066 [==============================] - 15s 14ms/step - loss: 0.7274 - accuracy: 0.7838
Epoch 4/30
1066/1066 [==============================] - 16s 15ms/step - loss: 0.6694 - accuracy: 0.8017
Epoch 5/30
1066/1066 [==============================] - 15s 14ms/step - loss: 0.6160 - accuracy: 0.8202
Epoch 6/30
1066/1066 [==============================] - 15s 14ms/step - loss: 0.5770 - accuracy: 0.8312
Epoch 7/30
1066/1066 [==============================] - 15s 14ms/step - loss: 0.5424 - accuracy: 0.8409
Epoch 8/30
1066/1066 [==============================] - 16s 15ms/step - loss: 0.5179 - accuracy: 0.8484
Epoch 9/30
1066/1066 [==============================] - 16s 15ms/step - loss: 0.4982 - accuracy: 0.8533
Epoch 10/30
1066/1066 [==============================] - 16s 15m

epoch,29
loss,0.33287
accuracy,0.90209
_runtime,3656
_timestamp,1620740504
_step,30
val_accuracy,10.62115


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,▁▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂█
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂█
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
val_accuracy,▁


...

epoch,29
loss,0.01205
accuracy,0.9963
_runtime,4733
_timestamp,1620820601
_step,30
val_accuracy,39.37885


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▄▅▆▇▇████████████████████████
_runtime,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃█
_timestamp,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃█
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
val_accuracy,▁


wandb: Agent Starting Run: p7bbego1 with config:
wandb: 	cell_type: rnn
wandb: 	dropout: 0.5
wandb: 	embedding_size: 128
wandb: 	epochs: 30
wandb: 	latent_dimension: 64
wandb: 	n_decoder_layers: 2
wandb: 	n_encoder_layers: 2


Epoch 1/30
1066/1066 [==============================] - 112s 103ms/step - loss: 0.9970 - accuracy: 0.7386
Epoch 2/30
1066/1066 [==============================] - 107s 100ms/step - loss: 0.6403 - accuracy: 0.8154
Epoch 3/30
1066/1066 [==============================] - 111s 104ms/step - loss: 0.5961 - accuracy: 0.8276
Epoch 4/30
1066/1066 [==============================] - 109s 103ms/step - loss: 0.5702 - accuracy: 0.8346
Epoch 5/30
1066/1066 [==============================] - 109s 102ms/step - loss: 0.5547 - accuracy: 0.8385
Epoch 6/30
1066/1066 [==============================] - 103s 97ms/step - loss: 0.5425 - accuracy: 0.8417
Epoch 7/30
1066/1066 [==============================] - 109s 102ms/step - loss: 0.5331 - accuracy: 0.8441
Epoch 8/30
1066/1066 [==============================] - 109s 102ms/step - loss: 0.5235 - accuracy: 0.8468
Epoch 9/30
1066/1066 [==============================] - 103s 96ms/step - loss: 0.5173 - accuracy: 0.8484
Epoch 10/30
1066/1066 [=========================

wandb: Ctrl + C detected. Stopping sweep.
